# 2.3 Symbolic Data

All the compound data objects we have used so far were constructed ultimately from numbers. In this section we extend the representational capability of our language by introducing the ability to work with arbitrary symbols as data.

## 2.3.1 Quotation

If we can form compound data using symbols, we can have lists such as

Lists containing symbols can look just like the expressions of our language:

In order to manipulate symbols we need a new element in our language:the ability to $\textbf{quote}$ a data object. Suppose we want to construct the list $\textrm{(a b)}$. We can’t accomplish this with $\textrm{(list a b)}$, because this expression constructs a list of the values of $a$ and $b$ rather than the symbols themselves. This issue is well known in the context of natural languages,where words and sentences may be regarded either as semantic entities or as character strings (syntactic entities). The common practice in natural languages is to use quotation marks to indicate that a word or a sentence is to be treated literally as a string of characters. For instance,the first letter of “John” is clearly “J.” If we tell somebody “say your name aloud,” we expect to hear that person’s name. However, if we tell somebody “say ‘your name’ aloud,” we expect to hear the words “your name.” Note that we are forced to nest quotation marks to describe what somebody else might say.

We can follow this same practice to identify $\text{lists}$ and $\text{symbols}$ that are to be treated as data objects rather than as expressions to be evaluated. However, our format for quoting differs from that of natural languages in that we place a quotation mark (traditionally, the single quote symbol $\textbf{'}$) only at the beginning of the object to be quoted. We can get away with this in Scheme syntax because we rely on blanks and parentheses to delimit objects. Thus, the meaning of the single quote character is to quote the next object.

Now we can $\text{distinguish}$ between symbols and their values:

Quotation also allows us to type in compound objects, using the conventional printed representation for lists:

In keeping with this, we can obtain the empty list by evaluating $\textrm{'()}$, and thus dispense with the variable $\text{nil}$.

## Example 14: Just an example

One additional primitive used in manipulating symbols is $\textrm{eq?}$, which takes two symbols as arguments and tests whether they are the same. Using $\textrm{eq?}$, we can implement a useful procedure called $\texttt{memq}$. This takes two arguments, a symbol and a list. If the symbol is not contained in the list (i.e., is not $\textrm{eq?}$ to any item in the list), then $\texttt{memq}$ returns false. Otherwise, it returns the sublist of the list beginning with the first occurrence of the symbol:

In [1]:
cat 2.3/Example_14/memq.scm

(define (memq item x)
  (cond ((null? x) #f)
        ((eq? item (car x)) x)
        (else (memq item (cdr x)))))


For example, the value of

## Exercise 2.53: 
What would the interpreter print in response to evaluating each of the following expressions?

## Answer:

## Exercise 2.54: 
Two lists are said to be equal? if they contain equal elements arranged in the same order. For example,

is true, but

is false. To be more precise, we can define $\texttt{equal?}$ recursively in terms of the basic $\texttt{eq?}$ equality of symbols by saying that $a$ and $b$ are $\texttt{equal?}$ if they are both symbols and the symbols are $\texttt{eq?}$, or if they are both lists such that $\textrm{(car a)}$ is $\texttt{equal?}$ to $\textrm{(car b)}$ and $\textrm{(cdr a)}$ is $\texttt{equal?}$ to $\textrm{(cdr b)}$.Using this idea, implement $\texttt{equal?}$ as a procedure.

## Answer:

In [1]:
cat 2.3/Exercise_2.54/equal.scm

(define (symbol-equal? x y) (eq? x y))
(define (list-equal? x y)
  (cond ((and (null? x) (null? y)) #t)
        ((or (null? x) (null? y)) #f)
        ((equal? (car x) (car y))
         (equal? (cdr x) (cdr y)))
        (else #f)))

(define (equal? x y)
  (cond ((and (symbol? x) (symbol? y))
         (symbol-equal? x y))
        ((and (list? x) (list? y))
         (list-equal? x y))
        (else (error "Wrong type input x and y..." x y))))


### Running Instance:

## Exercise 2.55: 
Eva Lu Ator types to the interpreter the expression 

To her surprise, the interpreter prints back quote. Explain.

## Answer:

Because the expression

is equivalent to

So,the interpreter prints:

## Example 15: Symbolic Differentiation

As an illustration of symbol manipulation and a further illustration of data abstraction, consider the design of a procedure that performs symbolic differentiation of algebraic expressions. We would like the procedure to take as arguments an algebraic expression and a variable and to return the $\text{derivative}$ of the expression with respect to the variable. For example, if the arguments to the procedure are $ax^2 + bx + c$ and $x$, the procedure should return $2ax + b$. Symbolic differentiation is of special historical significance in Lisp. It was one of the motivating examples behind the development of a computer language for symbol manipulation. Furthermore, it marked the beginning of the line of research that led to the development of powerful systems for symbolic mathematical work, which are currently being used by a growing number of applied mathematicians and physicists.

In developing the symbolic-differentiation program, we will follow the same strategy of data abstraction that we followed in developing the rational-number system of Section 2.1.1. That is, we will first define a differentiation algorithm that operates on abstract objects such as “sums,” “products,” and “variables” without worrying about how these are to be represented. Only afterward will we address the representation problem.

### $\spadesuit\quad$ The differentiation program with abstract data

In order to keep things simple, we will consider a very simple symbolic-differentiation program that handles expressions that are built up using only the operations of addition and multiplication with two arguments.
Differentiation of any such expression can be carried out by applying the following reduction rules:

\begin{align}
\frac{dc}{dx} & = 0,\quad\textrm{for c a constant or a variable different from x},\\
\frac{dx}{dx} & = 1,\\
\frac{d(u+v)}{dx} & = \frac{du}{dx}+\frac{dv}{dx},\\
\frac{d(uv)}{dx} & = u\frac{dv}{dx}+v\frac{du}{dx}.
\end{align}

Observe that the latter two rules are recursive in nature. That is, to obtain the derivative of a sum we first find the derivatives of the terms and add them. Each of the terms may in turn be an expression that needs to be decomposed. Decomposing into smaller and smaller pieces will eventually produce pieces that are either constants or variables, whose derivatives will be either 0 or 1.

To embody these rules in a procedure we indulge in a little wishful thinking, as we did in designing the rational-number implementation.If we had a means for representing algebraic expressions, we should be able to tell whether an expression is a sum, a product, a constant, or a variable. We should be able to extract the parts of an expression.For a sum, for example we want to be able to extract the addend (first term) and the augend (second term). We should also be able to construct expressions from parts. Let us assume that we already have procedures to implement the following $\text{selectors}$, $\text{constructors}$, and $\text{predicates}$:

\begin{align}
\textrm{(variable? e)}&\qquad\text{Is e a variable?}\\
\textrm{(same-variable? v1 v2)}&\qquad\text{Are v1 and v2 the same variable?}\\
\textrm{(sum? e)}&\qquad\text{Is e a sum?}\\
\textrm{(addend e)}&\qquad\text{Addend of the sum e.}\\
\textrm{(augend e)}&\qquad\text{Augend of the sum e.}\\
\textrm{(make-sum a1 a2)}&\qquad\text{Construct the sum of a1 and a2.}\\
\textrm{(product? e)}&\qquad\text{Is e a product?}\\
\textrm{(multiplier e)}&\qquad\text{Multiplier of the product e.}\\
\textrm{(multiplicand e)}&\qquad\text{Multiplicand of the product e.}\\
\textrm{(make-product m1 m2)}&\qquad\text{Construct the product of m1 and m2.}
\end{align}

Using these, and the primitive predicate $\texttt{number?}$, which identifies numbers,we can express the differentiation rules as the following procedure:

In [24]:
cat 2.3/Example_15/deriv-rules.scm

(load "2.3/Example_15/deriv-repr.scm")

(define (deriv expression var)
  (cond ((number? expression) 0)
        ((variable? expression) (if (same-variable? expression var) 1 0))
        ((sum? expression) (make-sum (deriv (addend expression) var)
                                     (deriv (augend expression) var)))
        ((product? expression)
         (make-sum
           (make-product (multiplier expression)
                         (deriv (multiplicand expression) var))
           (make-product (deriv (multiplier expression) var)
                         (multiplicand expression))))
        (else
          (error "Unknown expression type: DERIV" expression))))


This $\texttt{deriv}$ procedure incorporates the complete differentiation algorithm.Since it is expressed in terms of abstract data, it will work no matter how we choose to represent algebraic expressions, as long as we
design a proper set of selectors and constructors. This is the issue we must address next.

### $\spadesuit\quad$ Representing algebraic expressions

We can imagine many ways to use list structure to represent algebraic expressions. For example, we could use lists of symbols that mirror the usual algebraic notation, representing $\textrm{ax + b}$ as the list $\textrm{(a $\ast$ x + b)}$. However, one especially straightforward choice is to use the same parenthesized prefix notation that Lisp uses for combinations; that is,to represent $\textrm{ax + b}$ as $\textrm{(+ ($\ast$ a x) b)}$. Then our data representation for the differentiation problem is as follows:

$\bullet\quad$The variables are symbols. They are identified by the primitive predicate $\texttt{symbol?}$:

In [16]:
cat 2.3/Example_15/is-variable.scm

(define (variable? x) (symbol? x))


$\bullet\quad$Two variables are the same if the symbols representing them are $\texttt{eq?}$:

In [23]:
cat 2.3/Example_15/is-same-variable.scm

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))


$\bullet\quad$Sums are constructed as lists:

In [6]:
cat 2.3/Example_15/make-sum.scm

(define (make-sum a1 a2) (list '+ a1 a2))


$\bullet\quad$Products are constructed as lists:

In [7]:
cat 2.3/Example_15/make-product.scm

(define (make-product m1 m2) (list '* m1 m2))


$\bullet\quad$A sum is a list whose first element is the symbol +:

In [14]:
cat 2.3/Example_15/is-sum.scm

(define (sum? x) (and (pair? x) (eq? (car x) '+)))


$\bullet\quad$The addend is the second item of the sum list:

In [9]:
cat 2.3/Example_15/addend.scm

(define (addend s) (cadr s))


$\bullet\quad$The augend is the third item of the sum list:

In [10]:
cat 2.3/Example_15/augend.scm

(define (augend s) (caddr s))


$\bullet\quad$A product is a list whose first element is the symbol $\ast$:

In [15]:
cat 2.3/Example_15/is-product.scm

(define (product? x) (and (pair? x) (eq? (car x) '*)))


$\bullet\quad$The multiplier is the second item of the product list:

In [12]:
cat 2.3/Example_15/multiplier.scm

(define (multiplier p) (cadr p))


$\bullet\quad$The multiplicand is the third item of the product list:

In [13]:
cat 2.3/Example_15/multiplicand.scm

(define (multiplicand p) (caddr p))


Thus, we need only combine these with the algorithm as embodied by $\texttt{deriv}$ in order to have a working symbolic-differentiation program.

Let us look at some examples of its behavior:

The program produces answers that are correct; however, they are unsimplified.It is true that

$$\frac{d(xy)}{dx}=x \cdot 0 + 1 \cdot y,$$

but we would like the program to know that $x \cdot 0 = 0$, $1 \cdot y = y$, and $0 \cdot y = y$. The answer for the second example should have been simply $y$. As the third example shows, this becomes a serious issue when the
expressions are complex.

Our difficulty is much like the one we encountered with the rational-number implementation: we haven’t reduced answers to simplest form.To accomplish the rational-number reduction, we needed to change only the $\text{constructors}$ and the $\text{selectors}$ of the implementation. We can adopt a similar strategy here. We won’t change $\texttt{deriv}$ at all. Instead, we will change $\texttt{make-sum}$ so that if both summands are numbers, $\texttt{make-sum}$ will add them and return their sum. Also, if one of the summands is 0, then $\texttt{make-sum}$ will return the other summand.

In [18]:
cat 2.3/Example_15/make-sum-reduce.scm

(define (make-sum a1 a2)
  (cond ((is-number? a1 0) a2)
        ((is-number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2))))


This uses the procedure $\texttt{is-number?}$,which checks whether an expression is equal to a given number:

In [19]:
cat 2.3/Example_15/is-number.scm

(define (is-number? expression num)
  (and (number? expression) (= expression num)))


Similarly, we will change $\texttt{make-product}$ to build in the rules that 0 times anything is 0 and 1 times anything is the thing itself:

In [20]:
cat 2.3/Example_15/make-product-reduce.scm

(define (make-product m1 m2)
  (cond ((is-number? m1 1) m2)
        ((is-number? m2 1) m1)
        ((or (is-number? m1 0) (is-number? m2 0)) 0)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))


For simplicity, we will put all the $\text{constructors}$, $\text{selections}$ and $\text{predicates}$ above given together:

In [25]:
cat 2.3/Example_15/deriv-repr.scm

;;
;; Constructors
;;

;; Construct the sum of a1 and a2
(define (make-sum a1 a2)
  (cond ((is-number? a1 0) a2)
        ((is-number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2))))

;; Construct the product of m1 and m2
(define (make-product m1 m2)
  (cond ((is-number? m1 1) m2)
        ((is-number? m2 1) m1)
        ((or (is-number? m1 0) (is-number? m2 0)) 0)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))


;;
;; Selectors
;;

;; Addend of the sum s
(define (addend s) (cadr s))

;; Augend of the sum s
(define (augend s) (caddr s))

;; Multiplier of the product p
(define (multiplier p) (cadr p))

;; Multiplicand of the product p
(define (multiplicand p) (caddr p))

;;
;; Predicates
;;

;; Is num number?
(define (is-number? expression num)
  (and (number? expression) (= expression num)))

;; Is x variable?
(define (variable? x) (symbol? x))

;; Are v1 and v2 the same variable?
(define (same-variable? 

Here is how this version works on our three examples:

Although this is quite an improvement, the third example shows that there is still a long way to go before we get a program that puts expressions into a form that we might agree is “simplest.” The problem of algebraic simplification is complex because, among other reasons, a form that may be simplest for one purpose may not be for another.

## Exercise 2.56: 
Show how to extend the basic differentiator to handle more kinds of expressions. For instance, implement the differentiation rule

$$\frac{d(u^n)}{dx}=nu^{n-1}\frac{du}{dx}$$

by adding a new clause to the $\texttt{deriv}$ program and defining appropriate procedures $\texttt{xponentiation?}$, $\texttt{base}$, $\texttt{exponent}$,and $\texttt{make-exponentiation}$. (You may use the symbol ${**}$ to denote exponentiation.) Build in the rules that anything raised to the power 0 is 1 and anything raised to the power 1 is the thing itself.

## 解答：

$\spadesuit\quad$对Example_15中的$\text{deriv-rules.scm}$新增：

In [26]:
cat 2.3/Exercise_2.56/deriv-rules.scm

(load "2.3/Exercise_2.56/deriv-repr.scm")

(define (deriv expression var)
  (cond ((number? expression) 0)
        ((variable? expression) (if (same-variable? expression var) 1 0))
        ((sum? expression) (make-sum (deriv (addend expression) var)
                                     (deriv (augend expression) var)))
        ((product? expression)
         (make-sum
           (make-product (multiplier expression)
                         (deriv (multiplicand expression) var))
           (make-product (deriv (multiplier expression) var)
                         (multiplicand expression))))
        ;; START NEWLINES
        ((is-exponentiation? expression)
         (let ((n (exponent expression))
               (u (base expression)))
           (make-product n (make-product (make-exponentiation u (- n 1))
                                         (deriv u var)))))
        ;; END NEWLINES
        (else
          (error "Unknown expression type: DERIV" expression))))


$\spadesuit\quad$对Example_15中的$\text{deriv-repr.scm}$新增：

In [27]:
cat 2.3/Exercise_2.56/deriv-repr.scm

;;
;; Constructors
;;

;; Construct the sum of a1 and a2
(define (make-sum a1 a2)
  (cond ((is-number? a1 0) a2)
        ((is-number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2))))

;; Construct the product of m1 and m2
(define (make-product m1 m2)
  (cond ((is-number? m1 1) m2)
        ((is-number? m2 1) m1)
        ((or (is-number? m1 0) (is-number? m2 0)) 0)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))


;;
;; Selectors
;;

;; Addend of the sum s
(define (addend s) (cadr s))

;; Augend of the sum s
(define (augend s) (caddr s))

;; Multiplier of the product p
(define (multiplier p) (cadr p))

;; Multiplicand of the product p
(define (multiplicand p) (caddr p))

;;
;; Predicates
;;

;; Is num number?
(define (is-number? expression num)
  (and (number? expression) (= expression num)))

;; Is x variable?
(define (variable? x) (symbol? x))

;; Are v1 and v2 the same variable?
(define (same-variable? 

### Running Instance:

## Exercise 2.57: 
Extend the differentiation program to handle sums and products of arbitrary numbers of (two or more) terms. Then the last example above could be expressed as

Try to do this by changing only the representation for $\text{sums}$ and $\text{products}$, without changing the $\texttt{deriv}$ procedure at all.For example, the $\texttt{addend}$ of a sum would be the first term,and the $\texttt{augend}$ would be the sum of the rest of the terms.

$\spadesuit\quad$重新定义$\text{sum}$的生成器和选择器过程：

In [28]:
cat 2.3/Exercise_2.57/sum-in-deriv.scm

(define (make-sum a1 . a2)
  (if (single-operand? a2)
    (let ((a2 (car a2)))
      (cond ((is-number? a1 0) a2)
            ((is-number? a2 0) a1)
            ((and (number? a1) (number? a2)) (+ a1 a2))
            (else (list '+ a1 a2))))
    (cons '+ (cons a1 a2))))

(define (addend s) (cadr s))

(define (augend s)
  (let ((tail-operand (cddr s)))
    (if (single-operand? tail-operand)
        (car tail-operand)
        (apply make-sum tail-operand))))

(define (sum? x)
  (and (pair? x) (eq? (car x) '+)))


$\spadesuit\quad$重新定义$\text{product}$的生成器和选择器过程：

In [29]:
cat 2.3/Exercise_2.57/product-in-deriv.scm

(define (make-product m1 . m2)
  (if (single-operand? m2)
    (let ((m2 (car m2)))
      (cond ((or (is-number? m1 0) (is-number? m2 0)) 0)
            ((is-number? m1 1) m2)
            ((is-number? m2 1) m1)
            ((and (number? m1) (number? m2)) (* m1 m2))
            (else (list '* m1 m2))))
    (cons '* (cons m1 m2))))

(define (multiplier p) (cadr p))

(define (multiplicand p)
  (let ((tail-operand (cddr p)))
    (if (single-operand? tail-operand)
        (car tail-operand)
        (apply make-product tail-operand))))

(define (product? x)
  (and (pair? x) (eq? (car x) '*)))



其中的$\texttt{single-operand?}$过程将检查第二个参数是否只有单个操作符：

In [30]:
cat 2.3/Exercise_2.57/is-single-operand.scm

(define (single-operand? x)
  (not (pair? (cdr x))))


### Running Instance:

$\bullet\quad$首先导入Exercise_2.56中的$\texttt{deriv-rules}$过程：

$\bullet\quad$然后导入本题中的重新定义的$\texttt{sum}$与$\texttt{product}$构造器与选择器过程覆盖Exercise_2.56中导入的对应过程：

$\bullet\quad$运行表达式$\textrm{(deriv '(* x y (+ x 3)) 'x)}$进行验证：

一个较复杂的表达式的求值：

## Exercise 2.58: 
Suppose we want to modify the differentiation program so that it works with ordinary mathematical notation, in which $\text{+}$ and $\ast$ are infix rather than prefix operators.Since the differentiation program is defined in terms of abstract data, we can modify it to work with different representations of expressions solely by changing the $\text{predicates}$,$\text{selectors}$, and $\text{constructors}$ that define the representation of the algebraic expressions on which the differentiator is to operate.

$\quad a.\;$Show how to do this in order to differentiate algebraic expressions presented in infix form, such as $\textrm{(x + (3 * (x + (y + 2))))}$. To simplify the task, assume that $\text{+}$ and $\ast$ always take two arguments and that expressions are fully parenthesized.

$\quad b.\;$The problem becomes substantially harder if we allow standard algebraic notation, such as $\textrm{(x + 3 * (x + y + 2))}$, which drops unnecessary parentheses and assumes that multiplication is done before addition. Can you design appropriate predicates, selectors, and constructors for this notation such that our derivative program still works?